# Collaborative Filtering
> Author: [Yalim Demirkesen](github.com/demirkeseny) 

> Date: March 2019

In [1]:
# Necessary libraries:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import GridSearchCV, cross_val_score
import networkx as nx
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4

## Preprocessing

In [2]:
# Uploading the ratings.csv file:
ratings = pd.read_csv('./data/ratings.csv')

In [3]:
# Since one user can vote a book more than once, I need to group them
# and take the mode of all the votes so that I can generate one number.
ratings = ratings.groupby(['book_id','user_id']).agg(lambda x: stats.mode(x)[0][0]).reset_index()

Now we need to calculate how many times a certain user rated a book. It is important since we can only give a more accurate recommendations to users with higher amount of ratings. The more a user rates, more information we have about that person's taste.

In [4]:
# Create a series with the user ID's. 
v = ratings[['user_id']]
c = v.apply(pd.Series.value_counts)

# By resetting the index, I can add the ID as a new column:
c.reset_index(inplace=True)

# Renaming the columns as ID and occurance:
c.columns = ['id','occurance']

In [5]:
# Let's check how many rows our new data frame has.
# This number corresponds to each user and number
# of ratings:
c.shape

(53424, 2)

Since more the number of rating per user gets, better recommendations we can make, I will exclude all the users with less than 5 ratings.

In [6]:
c = c[c['occurance'] > 5]

In [7]:
# Creating a list with the ID's of all user that voted more than 5:
ids = c.id.tolist()

In [8]:
# Creating a new ratings list with the selected users:
ratings = ratings[ratings.user_id.isin(ids)]

In [9]:
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [10]:
# Checking the number of rows left in the ratings dataframe:
ratings.shape

(914505, 3)

In [11]:
print('There are ' + str(ratings.book_id.nunique()) + ' different books.')
print('There are ' + str(ratings.user_id.nunique()) + ' different users.')
print('There are ' + str(ratings[['book_id','user_id']].drop_duplicates().shape[0]) + ' unique user ratings.')

There are 10000 different books.
There are 32439 different users.
There are 914505 unique user ratings.


Now we need to upload our more detailed csv file that includes all the descriptions. Since there are also other languages than English, I need to use `encoding` function while uploading our data.

In [12]:
books_ext = pd.read_csv('./data/books_extended.csv', encoding='utf-8-sig')

In [13]:
books = pd.read_csv('./data/books.csv', encoding='utf-8-sig')

Merging these two datasets so that we can have the most extensive dataset about our books:

In [14]:
book = pd.merge(books[['id','book_id']], books_ext,
                      on='book_id', how = 'inner')

In [15]:
# Dropping the unnecessary columns:
book.drop(columns = ['book_id','Unnamed: 0', 'image_url', 'small_image_url'], 
           inplace = True)

In [16]:
# Just to have the same column names, I change the column 'id' to 'book_id':
book.rename(columns={'id':'book_id'}, inplace = True)

In [17]:
book.head()

,book_id,books_count,isbn,authors,original_publication_year,original_title,title_x,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description
0,1,272,439023483,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,"Could you survive on your own, in the wild, wi..."
1,2,491,439554934,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,Harry Potter's life is miserable. His parents ...
2,3,226,316015849,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,<b>About three things I was absolutely positiv...
3,4,487,61120081,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,The unforgettable novel of a childhood in a sl...
4,5,1356,743273567,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,Alternate Cover Edition ISBN: 0743273567 (ISBN...


After being left with the only necessary columns, now I make simple iterations on the books data frame. For instance, below we can see the books with the highest average ratings and the books that are rated at most. 

The winners in both fields are:
- The Complete Calvin and Hobbes by Bill Watterson with the average rating 4.82
- The Hunger Games (First Book of the Trilogy) by Suzanne Collins with the ratings count 4,780,653

In [18]:
book.sort_values(by=['average_rating'], ascending = False)[['authors','title_x','average_rating','ratings_count']].head(10)

,authors,title_x,average_rating,ratings_count
3627,Bill Watterson,The Complete Calvin and Hobbes,4.82,28900
3274,"J.K. Rowling, Mary GrandPré","Harry Potter Boxed Set, Books 1-5 (Harry Potte...",4.77,33220
861,Brandon Sanderson,"Words of Radiance (The Stormlight Archive, #2)",4.77,73572
8853,Francine Rivers,Mark of the Lion Trilogy,4.76,9081
7946,"Anonymous, Lane T. Dennis, Wayne A. Grudem",ESV Study Bible,4.76,8953
4482,Bill Watterson,It's a Magical World: A Calvin and Hobbes Coll...,4.75,22351
6360,Bill Watterson,There's Treasure Everywhere: A Calvin and Hobb...,4.74,16766
421,J.K. Rowling,"Harry Potter Boxset (Harry Potter, #1-7)",4.74,190050
3752,J.K. Rowling,"Harry Potter Collection (Harry Potter, #1-6)",4.73,24618
6919,Bill Watterson,The Indispensable Calvin and Hobbes,4.73,14597


In [19]:
book.sort_values(by=['ratings_count'], ascending = False)[['authors','title_x','average_rating','ratings_count']].head(10)

,authors,title_x,average_rating,ratings_count
0,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",4.34,4780653
1,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,4.44,4602479
2,Stephenie Meyer,"Twilight (Twilight, #1)",3.57,3866839
3,Harper Lee,To Kill a Mockingbird,4.25,3198671
4,F. Scott Fitzgerald,The Great Gatsby,3.89,2683664
5,John Green,The Fault in Our Stars,4.26,2346404
6,J.R.R. Tolkien,The Hobbit,4.25,2071616
7,J.D. Salinger,The Catcher in the Rye,3.79,2044241
9,Jane Austen,Pride and Prejudice,4.24,2035490
8,Dan Brown,"Angels & Demons (Robert Langdon, #1)",3.85,2001311


## Building the Recommendation Engine - Item Based

At this part, we will build a recommendation engine based on the items. In this algorithm, we use the user ratings and generate the most similar books.

We need to start by arranging our data in such a way that each individual rating has one row. 

So in order to do that we have a data frame consisting of:
- book ID
- title
- user ID
- rating!

In [20]:
rated_book = pd.merge(book[['book_id','title_x']], ratings,
                      on='book_id', how = 'inner')

In [21]:
rated_book.shape

(914505, 4)

In [22]:
rated_book.head()

,book_id,title_x,user_id,rating
0,1,"The Hunger Games (The Hunger Games, #1)",314,5
1,1,"The Hunger Games (The Hunger Games, #1)",439,3
2,1,"The Hunger Games (The Hunger Games, #1)",588,5
3,1,"The Hunger Games (The Hunger Games, #1)",1169,4
4,1,"The Hunger Games (The Hunger Games, #1)",1185,4


In [23]:
# Just to check whether we have 10,000 books in the new data frame:
rated_book.book_id.nunique()

10000

Below we create a pivot table. This is necessary to start any type of collaborative filtering. It is basically a matrix with as many rows as the number of books and as many columns as number of users. Since a user might have rated usually not more than 10 or 20 books, the rest is cells in the column of a user is left as `NaN`. 

In [24]:
pivot = pd.pivot_table(rated_book, index='title_x', columns='user_id', values='rating')
pivot.head()

user_id,7,9,10,11,16,18,19,22,23,24,...,53407,53409,53411,53413,53414,53419,53420,53421,53422,53424
title_x,,,,,,,,,,,,,,,,,,,,,
"Angels (Walsh Family, #3)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#GIRLBOSS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"'Tis (Frank McCourt, #2)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pivot.shape

(9964, 32439)

The missing values will be replaced with zero since they represent the lack of vote. This is not user voting it as zero out of five stars. Then we use cosine similarity between the books. This helps us to group the similar books considering the ratings of the users. 

According the cosine similarity, two items are more similar as the value approaches to zero. To be more specific, on the diagonal the value is always zero since one book is same as itself and cannot be more similar. 

In [26]:
sparse_pivot = sparse.csr_matrix(pivot.fillna(0))

In [27]:
recommender = pairwise_distances(sparse_pivot, metric='cosine')

In [28]:
recommender_df = pd.DataFrame(recommender, columns=pivot.index, index=pivot.index)
recommender_df.head()

title_x,"Angels (Walsh Family, #3)","""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",#GIRLBOSS,'Salem's Lot,"'Tis (Frank McCourt, #2)","1,000 Places to See Before You Die",1/4 جرام,"10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works","100 Bullets, Vol. 1: First Shot, Last Call",100 Love Sonnets,...,محال,مخطوطة بن إسحاق: مدينة الموتى,نادي السيارات,هشت کتاب,هيبتا,واحة الغروب,يوتوبيا,ڤيرتيجو,キスよりも早く1 [Kisu Yorimo Hayaku 1] (Faster than a Kiss #1),美少女戦士セーラームーン新装版 1 [Bishōjo Senshi Sailor Moon Shinsōban 1]
title_x,,,,,,,,,,,,,,,,,,,,,
"Angels (Walsh Family, #3)",0.000000,1.0,0.997434,1.000000,0.990435,0.993378,1.00000,1.000000,1.0,1.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",1.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.81221,1.000000,1.0,1.000000,...,0.836783,0.440869,0.795892,1.0,0.696079,0.799942,0.803693,0.741463,1.0,1.000000
#GIRLBOSS,0.997434,1.0,0.000000,1.000000,1.000000,1.000000,1.00000,0.938684,1.0,1.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,0.983575
'Salem's Lot,1.000000,1.0,1.000000,0.000000,0.992693,1.000000,1.00000,1.000000,1.0,0.991448,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000
"'Tis (Frank McCourt, #2)",0.990435,1.0,1.000000,0.992693,0.000000,0.980858,1.00000,1.000000,1.0,0.991910,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000


Below is one example for item based collaborative filtering. For instance when the book *Lost Symbol* by *Dan Brown* is searched we end up books either from the same author or same genre or similar context.

In [29]:
search = "Lost Symbol"

for title in book.loc[book['title_x'].str.contains(search), 'title_x']:
    print('Search:',title)
    print('Average rating:', pivot.loc[title, :].mean())
    print('Number of ratings:', pivot.T[title].count())
    print('')
    print('10 closest books are:')
    print(recommender_df[title].sort_values()[1:11])
    print('')

Search: The Lost Symbol (Robert Langdon, #3)
Average rating: 3.13
Number of ratings: 100

10 closest books are:
title_x
Digital Fortress                                          0.674100
Deception Point                                           0.693857
Inferno (Robert Langdon, #4)                              0.741665
The Girl Who Kicked the Hornet's Nest (Millennium, #3)    0.746636
Eclipse (Twilight, #3)                                    0.754782
The Runaway Jury                                          0.764390
A Dance with Dragons (A Song of Ice and Fire, #5)         0.767085
New Moon (Twilight, #2)                                   0.768256
Breaking Dawn (Twilight, #4)                              0.770408
The Da Vinci Code (Robert Langdon, #2)                    0.773650
Name: The Lost Symbol (Robert Langdon, #3), dtype: float64



## Building the Recommendation Engine - User Based

### Understanding User Based

After item based recommendation that generated similar books considering their distance with cosine similarity, it is time for generating a user based recommendation. The idea behind this approach is that we find the most similar users to our target user and then take the weighted average of the most similar users. The weight coefficient is higher if the similarity to target user increases.

In [30]:
pivot = pd.pivot_table(rated_book, index='user_id', columns='title_x', values='rating')
pivot.head()

title_x,"Angels (Walsh Family, #3)","""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",#GIRLBOSS,'Salem's Lot,"'Tis (Frank McCourt, #2)","1,000 Places to See Before You Die",1/4 جرام,"10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works","100 Bullets, Vol. 1: First Shot, Last Call",100 Love Sonnets,...,محال,مخطوطة بن إسحاق: مدينة الموتى,نادي السيارات,هشت کتاب,هيبتا,واحة الغروب,يوتوبيا,ڤيرتيجو,キスよりも早く1 [Kisu Yorimo Hayaku 1] (Faster than a Kiss #1),美少女戦士セーラームーン新装版 1 [Bishōjo Senshi Sailor Moon Shinsōban 1]
user_id,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
pivot.shape

(32439, 9964)

We will again create a sparse matrix but this time, the row and column number is equal to number of users that voted more than five times. 

In [32]:
sparse_pivot_user = sparse.csr_matrix(pivot.fillna(0))

In [33]:
sim_user = pairwise_distances(sparse_pivot_user, metric='cosine')

In [34]:
sim_user_df = pd.DataFrame(sim_user, columns=pivot.index, index=pivot.index)
sim_user_df.head()

user_id,7,9,10,11,16,18,19,22,23,24,...,53407,53409,53411,53413,53414,53419,53420,53421,53422,53424
user_id,,,,,,,,,,,,,,,,,,,,,
7,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0
9,1.0,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,0.954732,...,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0
10,1.0,1.0,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0
11,1.0,1.0,1.0,0.0,1.0,1.0,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0
16,1.0,1.0,1.0,1.0,0.0,1.0,0.891104,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,0.822482,1.0,1.0,1.0,1.0,1.0


Below, I used networkx module that enabled me to generate a dynamic undirected network graph. That was helpful to understand with which users there is more connection. To give an example, I picked the user with the ID number `12874`. 

The data frame that has three columns displays how similar the users to `12874` are. In order to do this, I sorted the `similarity` column from smallest to the highest. The scores in this column are multiplied with `100` in order to get rid of decimals. 

Again, the smallest this number gets, higher is the similarity!

In [35]:
user1 = sim_user_df[[12874]].sort_values(by=[12874]).reset_index().iloc[1:]
user1.columns = ['from','similarity']
user1['from'] = user1['from'].astype(str)
user1['to'] = '12874'
user1['similarity'] = (user1['similarity']*100).astype(int)
user1.head(20)

,from,similarity,to
1,12946,53,12874
2,36099,55,12874
3,45554,57,12874
4,21217,61,12874
5,48559,62,12874
6,47800,62,12874
7,32305,62,12874
8,49298,63,12874
9,33065,63,12874
10,3922,63,12874


After having this dataframe, I started constructing the network graph. That was helpful to see the connection of users to our target user.

In [36]:
# Initiating the networkx module by including the most similar 25 user in undirected graph.
G=nx.from_pandas_edgelist(user1.head(25), 'from', 'to', create_using=nx.DiGraph() )

In [37]:
# Degrees of similarity is introduced. Darker the color of the link gets, 
# higher the similarity is or lower the cosine similarity coefficient is.
deg1, deg2, deg3, deg4 = '#8B4513', '#A0522D', '#CD853F', '#DEB887'
edge_attrs = {}

In [38]:
for start_node, end_node, _ in G.edges(data=True):
    
    # created a mask for data frame. Essential to filter out the rows we need.
    mask = int(user1[(user1['from'] == start_node) & (user1['to'] == end_node)]['similarity'])

    # Assigned corresponding colors to each one of the nodes
    if (int(mask) > 50) & (int(mask) <58):
        edge_color = deg1
    elif (int(mask) > 57) & (int(mask) < 62):
        edge_color = deg2
    elif (int(mask) > 61) & (int(mask) < 64):
        edge_color = deg3
    else: 
        edge_color = deg4
    
    edge_attrs[(start_node, end_node)] = edge_color # assign the colors to the nodes.

In [39]:
nx.set_edge_attributes(G, edge_attrs, "edge_color")

In [40]:
# Show with Bokeh
plot = Plot(plot_width=400, plot_height=400,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Graph Interaction Demonstration"

node_hover_tool = HoverTool(tooltips=[("Node", "@index")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

# Listed all the necessary layout options that might match our visual preferences: 
# graph_renderer = from_networkx(G, nx.circular_layout, scale=1, center=(0, 0))
# graph_renderer = from_networkx(G, nx.random_layout, center = (-0.5,-0.5))
graph_renderer = from_networkx(G, nx.shell_layout,  center=(0, 0))
# graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.8, line_width=2)
plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)

<img src="images/users.PNG" alt="Drawing" style="width: 400px;"/>

So here we can see that all the links are initiating from our target users to the similar users. The similarity increases in clockwise until it reaches back to the target user. 

In [41]:
# Downloading the updated ratings:
ratings.to_csv('./data/ratings_updated.csv')